# CRM using Simpy

CRM system simulation using simpy. 

Company CRM populated with accounts (customer companies) and opportunities coming from time to time from these accounts and which can be bidded, won or lost by Company.

Principles: Account "agents" wich will go through a set of events, namely:
- inbound_mktg_event: account finds the contant and follows a CTA to be added into Company MQL list
- outbount_mkt_event: account reached by Company outbound marketing actions and follows a CTA to be added into Company MQL
- inbound_sales_event: account is identified as a sales qualified leads and converted into prospect
- outbound_sales_event: account reached out by Company sales representative and accepts call to become a prospect
and other similar events representing how an account moves from stage to stage through the sales funnel and sales process

Each event is the result of:
- the current phase where the account is 
- an adjustable random variable, using some paramaters (example, conversion rate for CTA, ...)

In [25]:
import random
import simpy

from eccore.ipython import nb_setup
nb_setup()

from agents import Stage, Account, CONVERSION_RATES, DELAY_RANGES, account_arrival, accounts_created_before


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Set autoreload mode


In [26]:
random.seed(1988)
env = simpy.Environment()
env.accounts = []
initial_accounts = 5
arrival_rate = 0.5  # on average, one new account every 2 units of time
env.process(account_arrival(env, initial_accounts, arrival_rate))
env.run(until=20)

[0.00] Company_2: Did not convert from marketing
[0.00] Company_3: Did not convert from marketing
[0.00] Company_3: Did not convert from marketing
[0.00] Company_5: Did not convert from marketing
[0.00] Company_5: Did not convert from marketing
[1.10] Company_2: Converted via inbound marketing to MQL
[1.10] Company_2: Did not convert from sales
[1.22] Company_4: Converted via inbound marketing to MQL
[1.99] Company_1: Converted via inbound marketing to MQL
[2.24] Company_1: Converted via inbound marketing to MQL
[2.55] Company_4: Converted via outbound marketing to MQL
[2.55] Company_4: Did not convert from sales
[3.35] Company_6: Did not convert from marketing
[3.35] Company_6: Did not convert from marketing
[4.86] Company_4: Converted via outbound sales to PROSPECT
[5.36] Company_1: Converted via outbound sales to PROSPECT
[5.57] Company_1: Converted via outbound sales to PROSPECT
[8.17] Company_7: Converted via inbound marketing to MQL
[8.17] Company_7: Did not convert from sales
[1

In [30]:
accounts_created_before(env=env, t=18)